## Generate Training Data for DeepCormack: PCA + DMD

#### Required:
- A DFT calculated 3D electron momentum density (copper in our case)

- Output:

In [ ]:
# Create conda environment for MCM first. In the terminal, run:
# conda env create -f environment.yml
from MCM_functions_new import *

In [4]:
"""Defining Modified Cormack Method Parameters"""

N = 513                                                                     # image size (N x N pixels) of copper projections                                    
raw_proj_dir = "Data_Generation_Required/Cu_20Projections(513)"             # directory containing ideal copper projections

pang = np.linspace(0, 45, 20)                                               # projection angles for the 20 projections for the DFT copper data        
ncoeffs = [150, 120, 100, 90, 80, 50]                                       # number of Chebyshev coefficients for each projection angle      
ncoeffs.extend([30]* (20-len(ncoeffs)))

order = 4                                                                   # copper's FCC structure means it has 4-fold symmetry (order = 4)                                  
nphi = 180                                                                  # angle increments for delta in polar unit circle are given by 90/nphi; 'm_max = nphi' when computing Chebyshev coefficients      
# calib = 0.09808                                                             # Experimental data calibration factor (pixels per mm) for copper data — NOT USED                           
xsize = (N + 1) // 2                                                        # Centre of the image in pixels
rhocut = 1                                                                  # 1 = don't cut off, 0 = cut off at flvl 
flvl = 100                                                                  # Choose Fermi level so the cutoff starts at your desired px (e.g., 256) (max value is N/2 = 256 for N=513)
kt = 6.0                                                                    # Slope; increase for smoother, decrease for sharper
rhofn = [rhocut, flvl, kt]

# Generating Experimental Data.
num_simulations = 1                                                         # Number of simulated 3D TPMD datasets to generate                               
count_ttl = 200_000_000                                                     # What should the simulated total counts be? ≈200,000,000 corresponds to 3 months of measurements!

base_dir = "Data_Generation_Required/TPMD_Data"
rho_real_dir = os.path.join(base_dir, "Rho_Measurement")
os.makedirs(rho_real_dir, exist_ok=True)

rho_ideal_dir = os.path.join(base_dir, "Rho_GroundTruth")
os.makedirs(rho_ideal_dir, exist_ok=True)




In [ ]:
"""--- Copper 3D TPMD ---"""
# Calculate a_n^m and rho_n for the ideal DFT copper projections. PCA per channel 'n' will be used on copper a_n^m 
# coefficients all 256 slices combined. Randomly sampling from each of the 'n' the latent spaces will generate new, 
# realistic a_n^m coefficients, giving us a consistent way to create new, realistic rho_n for TPMD central slices.

# Fetching the ideal copper projections
file_names = [f"I_TPMD2D.OUT_PROJ_{i+1}" for i in range(20)]
raw_projs = load_projections(raw_proj_dir, file_names, N)                   # shape (N, N, 20)


"""Run the loop below to generate 'num_simulations' realistic projections, their corresponding rho_n, and save them as .txt files."""


"""Only need to save one ideal rho_n, as it will be the same for all simulations (we're not adding noise/smearing to the ideal case):"""
# rhoreturn_ideal = getrho(raw_projs, order, 20, calib, pang, nphi, 20 * [120], rhofn, sinmat)    # Shape (256, 20, 256)


# getrho(raw_projs, order, 20, calib, pang, nphi, 20 * [120], rhofn, sinmat)
# def getrho(rawdat, order, pang, nphi, ncoeff, rhofn):
rhoreturn_ideal_Cu, anm_Cu = getrho(raw_projs, order, pang, nphi, 20 * [120], rhofn)
